In [13]:
# Install all relevant packages that are needed to run this Jupyter Notebook
%pip install anyio==4.4.0 \
argon2-cffi==23.1.0 \
argon2-cffi-bindings==21.2.0 \
arrow==1.3.0 \
asttokens==2.4.1 \
async-lru==2.0.4 \
attrs==24.2.0 \
babel==2.16.0 \
beautifulsoup4==4.12.3 \
bleach==6.1.0 \
certifi==2024.8.30 \
cffi==1.17.1 \
charset-normalizer==3.3.2 \
colorama==0.4.6 \
comm==0.2.2 \
debugpy==1.8.5 \
decorator==5.1.1 \
defusedxml==0.7.1 \
et-xmlfile==1.1.0 \
executing==2.1.0 \
fastjsonschema==2.20.0 \
fqdn==1.5.1 \
h11==0.14.0 \
httpcore==1.0.5 \
httpx==0.27.2 \
idna==3.10 \
ipykernel==6.29.5 \
ipython==8.27.0 \
ipywidgets==8.1.5 \
isoduration==20.11.0 \
jedi==0.19.1 \
Jinja2==3.1.4 \
json5==0.9.25 \
jsonpointer==3.0.0 \
jsonschema==4.23.0 \
jsonschema-specifications==2023.12.1 \
jupyter==1.1.1 \
jupyter-console==6.6.3 \
jupyter-events==0.10.0 \
jupyter-lsp==2.2.5 \
jupyter_client==8.6.3 \
jupyter_core==5.7.2 \
jupyter_server==2.14.2 \
jupyter_server_terminals==0.5.3 \
jupyterlab==4.2.5 \
jupyterlab_pygments==0.3.0 \
jupyterlab_server==2.27.3 \
jupyterlab_widgets==3.0.13 \
MarkupSafe==2.1.5 \
matplotlib-inline==0.1.7 \
mistune==3.0.2 \
nbclient==0.10.0 \
nbconvert==7.16.4 \
nbformat==5.10.4 \
nest-asyncio==1.6.0 \
notebook==7.2.2 \
notebook_shim==0.2.4 \
numpy==2.1.1 \
openpyxl==3.1.5 \
overrides==7.7.0 \
packaging==24.1 \
pandas==2.2.2 \
pandocfilters==1.5.1 \
parso==0.8.4 \
platformdirs==4.3.6 \
prometheus_client==0.20.0 \
prompt_toolkit==3.0.47 \
psutil==6.0.0 \
pure_eval==0.2.3 \
pycparser==2.22 \
Pygments==2.18.0 \
python-dateutil==2.9.0.post0 \
python-json-logger==2.0.7 \
pytz==2024.2 \
pywin32==306 \
pywinpty==2.0.13 \
PyYAML==6.0.2 \
pyzmq==26.2.0 \
referencing==0.35.1 \
requests==2.32.3 \
rfc3339-validator==0.1.4 \
rfc3986-validator==0.1.1 \
rpds-py==0.20.0 \
Send2Trash==1.8.3 \
six==1.16.0 \
sniffio==1.3.1 \
soupsieve==2.6 \
stack-data==0.6.3 \
terminado==0.18.1 \
tinycss2==1.3.0 \
tornado==6.4.1 \
traitlets==5.14.3 \
types-python-dateutil==2.9.0.20240906 \
typing_extensions==4.12.2 \
tzdata==2024.1 \
uri-template==1.3.0 \
urllib3==2.2.3 \
voila==0.5.7 \
wcwidth==0.2.13 \
webcolors==24.8.0 \
webencodings==0.5.1 \
websocket-client==1.8.0 \
websockets==13.1 \
widgetsnbextension==4.0.13


Note: you may need to restart the kernel to use updated packages.



[notice] A new release of pip is available: 24.0 -> 24.3.1
[notice] To update, run: python.exe -m pip install --upgrade pip


In [14]:
import pandas as pd
import numpy as np
import requests
import json


import requests
import pandas as pd
import ipywidgets as widgets
from datetime import datetime, timedelta
from IPython.display import display

In [15]:
API_ENDPOINT = "https://api.solar.sheffield.ac.uk/pvlive/api/v4" #Without parameters and any additional settings


second_to_third_map = {}
def fetch_gsp_list():
    """
    Fetch the list of GSPs available via the API.
    
    :return: A dictionary containing the list of GSPs, along with their index and ID
    """
    response = requests.get(f"{API_ENDPOINT}/gsp_list")
    if response.status_code == 200:
        return response.json()
    else:
        print(f"Failed to retrieve GSP list. Status code: {response.status_code}")
        return []
    


def fetch_solar_data(gsp, date1, date2, capacity_mw,oversize_amt, peak_mwp ):
    """
    Fetch half-hourly solar data for a specific GSP and date.
    
    :param gsp: The chosen GSP ID
    :param date1: The chosen start date
    :param date2: The chosen end date
    :param capacity_mw: The site capacity for the client, which is default set to 100 MW
    
    :return: The final dataframe, after the API call has been made and data has been processed
    """
    
    # Convert input dates to datetime objects
    start_date = datetime.strptime(date1, '%Y-%m-%d')
    end_date = datetime.strptime(date2, '%Y-%m-%d')

    # List to store all the data
    all_data = []

    # Calculate the duration between start and end dates
    duration = (end_date - start_date).days

    if duration < 0:
        print("End date must be after start date.")
        return None

    # Determine if we need to make a single or multiple API calls
    if duration <= 365:  # Single API call if duration is less than or equal to 1 year
        params = {
            "gsp": gsp,
            "start": start_date.strftime('%Y-%m-%d'),
            "end": end_date.strftime('%Y-%m-%d'),
            "date_format": 'csv',
            "extra_fields": 'capacity_mwp,installedcapacity_mwp',
            "period": '30'
        }
        
        # Log the API request
        print(f'gsp {gsp}')
        print(f'start {start_date.strftime("%Y-%m-%d")}')
        print(f'end {end_date.strftime("%Y-%m-%d")}')
        
        # Make the API call
        response = requests.get(f"{API_ENDPOINT}/gsp/{gsp}?start={date1}&end={date2}&data_format=json&extra_fields=capacity_mwp,installedcapacity_mwp&period=30")

        if response.status_code == 200:
            data = response.json()
            all_data.extend(data['data'][::-1])
            print(len(all_data))
        else:
            print(f"Error fetching data: {response.status_code}")
            return None

    else:  # Multiple API calls if the duration exceeds 1 year
        while start_date <= end_date:
            # Calculate the end date for this API call (max one year from start_date)
            current_end_date = min(start_date + timedelta(days=365), end_date)

            # Prepare API request parameters
            params = {
                "gsp": gsp,
                "start": start_date.strftime('%Y-%m-%d'),
                "end": current_end_date.strftime('%Y-%m-%d'),
                "date_format": 'csv',
                "extra_fields": 'capacity_mwp,installedcapacity_mwp',
                "period": '30'
            }

            # Log the API request
            print(f'gsp {gsp}')
            print(f'start {start_date.strftime("%Y-%m-%d")}')
            print(f'end {current_end_date.strftime("%Y-%m-%d")}')
            
            # Make the API call
            response = requests.get(f"{API_ENDPOINT}/gsp/{gsp}?start={start_date}&end={current_end_date}&data_format=json&extra_fields=capacity_mwp,installedcapacity_mwp&period=30")

            if response.status_code == 200:
                data = response.json()
                all_data.extend(data['data'][::-1])
                print(len(all_data))
            else:
                print(f"Error fetching data: {response.status_code}")
                return None

            # Move to the next date range (start from the day after current_end_date)
            start_date = current_end_date + timedelta(days=1)

    # Create a DataFrame from the collected data
    df = pd.DataFrame(all_data)

    # Process the DataFrame
    
        # Reverse the DataFrame
        # Save to Excel without row index
    
    rel_rows = all_data
    rel_head = data['meta']
    df = pd.DataFrame(rel_rows, columns=rel_head)
    df_reversed = df.iloc[::-1].reset_index(drop=True) # Reverse the output to go from start date to end date
    excel_filename = 'output_new_combined.xlsx'  # Name of the output file
    df_reversed['generation_mw'] = np.minimum((df_reversed['generation_mw']*((100+oversize_amt)/100)*peak_mwp/df_reversed['installedcapacity_mwp']),capacity_mw) # multiply the generation with our capacity/installed capacity and then scale by 20%
    df_reversed.to_excel(excel_filename, index=False)  # Save to Excel without row index
    print(f"Data has been saved to {excel_filename}")
        
    df_reversed['datetime_gmt'] = pd.to_datetime(df_reversed['datetime_gmt'],utc=True)
    df_reversed.set_index('datetime_gmt', inplace=True)


# Calculate the moving average over 2 half-hour intervals for all columns
    df_moving_avg = df_reversed.rolling(window=2).mean()

# Resample to hourly data, taking the mean of the moving averages
    hourly_data = df_moving_avg.resample('H').mean()
    hourly_data.index = hourly_data.index.tz_localize(None)

# Display the updated DataFrame
        
    hourly_data.to_excel('Hourly data_combined.xlsx', index=True)  # Save to Excel without row index
    print(f"Data has been saved to Hourly data_combined.xlsx")
        #df = pd.DataFrame([data])  # Wrap it in a list to create a DataFrame

    # Save the DataFrame to a CSV file
        #df.to_csv('output.csv', index=False)  # Save to CSV, without the index column
        #print("Data has been saved to output.csv")
    return df_reversed  # Optionally return the DataFrame

# Example usage
#df = fetch_solar_data('GSP_NAME', '2020-01-01', '2022-12-31', 10)  # Replace 'GSP_NAME' and other parameters as needed


def on_button_click(b):
    """
    Handle the button click event and fetch the data.
    
    :param b: event of pressing the "Fetch" button
    """
    selected_second_item = gsp_dropdown.value  # Get the selected second item
    selected_date1 = date_input1.value
    formatted_date1 = selected_date1.strftime('%Y-%m-%d')
    selected_date2 = date_input2.value
    formatted_date2 = selected_date2.strftime('%Y-%m-%d')
    corresponding_sublist = second_to_sublist_map.get(selected_second_item)  # Get the corresponding sublist
    corresponding_third_item = corresponding_sublist[2]
    capacity_mw = solar_capacity_input.value
    oversize_amt = oversize_percent.value
    peak_mwp = solar_peak_input.value
    data = fetch_solar_data(corresponding_third_item, formatted_date1, formatted_date2,capacity_mw,oversize_amt, peak_mwp)
    
    # Print the selected values (or you could store them in variables or a database)
    

# Create Dropdown menu for GSP selection
gsp_list = fetch_gsp_list()
second_to_sublist_map = {}

# Populate the mapping with second items as keys and the corresponding sublists as values
for value_list in gsp_list.values():
    for sublist in value_list:
        second_to_sublist_map[sublist[1]] = sublist
    
    break
            

# Create the dropdown options (display the second items)
dropdown_options = list(second_to_third_map.keys())

gsp_dropdown = widgets.Dropdown(
    options=list(second_to_sublist_map.keys()),  # Populate with second items
    description='Select GSP:',
    disabled=False
)

# Create Date input for selecting the time period
date_input1 = widgets.DatePicker(
    description='Pick a Start Date',
    disabled=False
)

date_input2 = widgets.DatePicker(
    description='Pick an end Date',
    disabled=False
)

solar_capacity_input = widgets.FloatText(
    value=100.0,  # Set the default value to 100
    description='Solar Capacity (MW):',
    placeholder='Enter total solar capacity',
    style={'description_width': 'initial'}
)

solar_peak_input = widgets.FloatText(
    value=100.0,  # Set the default value to 100
    description='Solar Peak Rating (MWp):',
    placeholder='Enter peak capacity',
    style={'description_width': 'initial'}
)

oversize_percent = widgets.FloatText(
    value=20.0,  # Set the default value to 100
    description='Oversize percentage:',
    placeholder='Enter oversize percentage (eg 20%)',
    style={'description_width': 'initial'}
)



In [18]:
# Button to trigger data fetch
fetch_button = widgets.Button(description="Fetch Data")
fetch_button.on_click(on_button_click)

# Display the widgets
display(gsp_dropdown, date_input1, date_input2, solar_capacity_input,oversize_percent, solar_peak_input, fetch_button)

Dropdown(description='Select GSP:', index=41, options=('NATIONAL', 'ABHA1', 'ABNE_P', 'ABTH_1', 'ACTL_2|CBNK_H…

DatePicker(value=datetime.date(2022, 1, 1), description='Pick a Start Date', step=1)

DatePicker(value=datetime.date(2023, 1, 1), description='Pick an end Date', step=1)

FloatText(value=100.0, description='Solar Capacity (MW):', style=DescriptionStyle(description_width='initial')…

FloatText(value=20.0, description='Oversize percentage:', style=DescriptionStyle(description_width='initial'))

...

Data has been saved to Hourly data_combined.xlsx
gsp 20
start 2022-01-01
end 2023-01-01
17521
Data has been saved to output_new_combined.xlsx


C:\Users\SiddharthPuri\AppData\Local\Temp\ipykernel_51068\2764606818.py:132: FutureWarning: 'H' is deprecated and will be removed in a future version, please use 'h' instead.
  hourly_data = df_moving_avg.resample('H').mean()


Data has been saved to Hourly data_combined.xlsx
gsp 20
start 2022-01-01
end 2023-01-01
17521
Data has been saved to output_new_combined.xlsx


C:\Users\SiddharthPuri\AppData\Local\Temp\ipykernel_51068\2764606818.py:132: FutureWarning: 'H' is deprecated and will be removed in a future version, please use 'h' instead.
  hourly_data = df_moving_avg.resample('H').mean()


Data has been saved to Hourly data_combined.xlsx
